In [1]:
## Preâmbulo
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.api as qqplot
import random
import yfinance as yf
import os

# bibiotecas Time_series
import matplotlib.dates as mdates
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima.model import ARIMAResults

import scipy.stats as stats

from statsmodels.tsa.holtwinters import ExponentialSmoothing

## Estatísticas de ajuste:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score

from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV, KFold

import xgboost as xgb

from sklearn.metrics import make_scorer
import numpy as np
from sklearn.model_selection import RandomizedSearchCV

# Outros modelos de regressão
from sklearn.ensemble import StackingRegressor, RandomForestRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor
from lightgbm import LGBMRegressor
from sklearn.neural_network import MLPRegressor


In [ ]:

df = pd.read_csv('C:/Users/jpzam/Desktop/CHALLENGE_PERSONAL/data/ativos_csv_2023.csv')

df.head()

df.set_index('Date', inplace= True)
df.index = pd.to_datetime(df.index, format = '%Y-%m-%d')


df.tail()

In [ ]:
## Importando Bitcoin; fazendo join em 'df'

bit_df = yf.download('BTC', start = '2023-01-03')

bit_df = bit_df[['Close']].rename(columns = {'Close':'BTC'})

bit_df.index = pd.to_datetime(bit_df.index, format = '%Y-%m-%d')

df.index = pd.to_datetime(df.index)

df_start = df.merge(bit_df, left_index=True, right_index=True, how = 'left').copy()

df_start.head()

In [ ]:
## Um XGBoost para cada ativos dos 61;

#for col in df.columns
def stock_predict(col, df):

    forecast_df = pd.DataFrame()

    df_col = df[[col]].dropna().copy()

    ## Fazendo o train-test split (10)

    len_df = len(df_col)
    train_size = int(len_df*0.8)

    #print(len_df)

    train = df_col.iloc[:train_size]
    train.index = pd.to_datetime(train.index)
    train = train.asfreq('D', method = 'ffill')

    test = df_col.iloc[train_size:]
    test.index = pd.to_datetime(test.index)
    test = test.asfreq('D', method = 'ffill')

    #print(train)
    #print(test)

    df_col = df_col.asfreq('D', method = 'ffill')

    merged_df = df_col.copy() #pd.merge(df_final, avir_df, how='left', right_index=True, left_index=True)

    merged_df.tail(10)

    ## Calculando as Trends


    def des_optimizer(train, alphas, betas, step=int(len(test))):
        best_alpha, best_beta, best_mae = None, None, float("inf")
        for alpha in alphas:
            for beta in betas:
                des_model = ExponentialSmoothing(train, trend="add").fit(smoothing_level=alpha, smoothing_trend=beta)
                y_pred = des_model.forecast(step)
                mae = mean_absolute_error(test, y_pred)
                if mae < best_mae:
                    best_alpha, best_beta, best_mae = alpha, beta, mae
                #print("alpha:", round(alpha, 2), "beta:", round(beta, 2), "mae:", round(mae, 4))
        print("best_alpha:", round(best_alpha, 2), "best_beta:", round(best_beta, 2), "best_mae:", round(best_mae, 4))
        return best_alpha, best_beta, best_mae

    alphas = np.arange(0.01, 1, 0.10)
    betas = np.arange(0.01, 1, 0.10)

    best_alpha, best_beta, best_mae = des_optimizer(train, alphas, betas)

    decomp = ExponentialSmoothing(merged_df[col].dropna(), trend='multiplicative')
    fit = decomp.fit(smoothing_level=0.11, smoothing_trend=0.01)
    merged_df['trend'] = fit.fittedvalues

    fcast = fit.forecast(25)

    merged_df[[col,'trend']].plot()

    fcast.plot() 

    ## Acoplando FCAST

    fcast.index.name = 'Date'

    fcast_df = pd.DataFrame(columns = merged_df.columns, index = fcast.index)
    fcast_df['trend'] = fcast.values
    fcast_df[col] = np.nan

    fcast_df.head()

    # Juntando no Merged_df
    merged_df = pd.concat([merged_df, fcast_df])

    #### CRIANDO AS FEATURES ####
    # ## Features 1 - Preços lags

    #merged_df['lag_3'] = merged_df['AVIR'].shift(3).fillna(method = 'ffill')
    merged_df['lag_5'] = merged_df[col].shift(5)
    merged_df['lag_7'] = merged_df[col].shift(7)
    merged_df['lag_10'] = merged_df[col].shift(10)
    merged_df['lag_15'] = merged_df[col].shift(15)
    merged_df['lag_30'] = merged_df[col].shift(30)


    ## Criando as Moving averages
    # Step 1: Calculate EMAs
    merged_df['EMA_9'] = merged_df[col].ewm(span=9, adjust=False).mean()

    # Step 2: Calculate SMAs
    merged_df['SMA_5'] = merged_df[col].rolling(window=5).mean()
    merged_df['SMA_15'] = merged_df[col].rolling(window=15).mean()
    merged_df['SMA_30'] = merged_df[col].rolling(window=30).mean()

    # Step 3: Calculate RSI
    def calculate_rsi(data, window=14):
        delta = data.diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
        rs = gain / loss
        rsi = 100 - (100 / (1 + rs))
        return rsi

    merged_df['RSI'] = calculate_rsi(merged_df[col], window=14)

    merged_df['RSI_lag_5'] = merged_df['RSI'].shift(5)
    merged_df['RSI_lag_7'] = merged_df['RSI'].shift(7)
    merged_df['RSI_lag_10'] = merged_df['RSI'].shift(10)
    merged_df['RSI_lag_15'] = merged_df['RSI'].shift(15)
    merged_df['RSI_lag_30'] = merged_df['RSI'].shift(30)

    # Step 4: Calculate MACD
    merged_df['EMA_12'] = merged_df[col].ewm(span=12, adjust=False).mean()
    merged_df['EMA_26'] = merged_df[col].ewm(span=26, adjust=False).mean()
    #merged_df['MACD'] = merged_df['EMA_12'] - merged_df['EMA_26']
    #merged_df['MACD_SIGNAL'] = merged_df['MACD'].ewm(span=9, adjust=False).mean()  # Signal line

        ## VAR PERCENTUAL DOS SMAs
    pct_5 = merged_df['SMA_5'].dropna().pct_change().iloc[-1]
    pct_15 = merged_df['SMA_15'].dropna().pct_change().iloc[-1]
    pct_30 = merged_df['SMA_30'].dropna().pct_change().iloc[-1]
    #pct_12 = merged_df['EMA_12'].dropna().pct_change().iloc[-1]
    #pct_26 = merged_df['EMA_26'].dropna().pct_change().iloc[-1]

    #for na in range(len(merged_df['SMA_30'].isna())):
    #    merged_df['EMA_12'] = np.where(merged_df['SMA_30'].isna(), np.nan, merged_df['EMA_12'])
    #for na in range(len(merged_df['SMA_30'].isna())):
    #    merged_df['EMA_12'] = merged_df['EMA_12'].fillna(merged_df['EMA_12'].shift() * (1 + pct_30))

    for na in range(len(merged_df['SMA_30'].isna())):
        merged_df['EMA_26'] = np.where(merged_df['SMA_30'].isna(), np.nan, merged_df['EMA_26'])
    for na in range(len(merged_df['SMA_30'].isna())):
        merged_df['EMA_26'] = merged_df['EMA_26'].fillna(merged_df['EMA_26'].shift() * (1 + pct_30))

    for na in range(len(merged_df['SMA_5'].isna())):
        merged_df['SMA_5'] = merged_df['SMA_5'].fillna(merged_df['SMA_5'].shift() * (1 + pct_5))
        
    for na in range(len(merged_df['SMA_15'].isna())):
        merged_df['SMA_15'] = merged_df['SMA_15'].fillna(merged_df['SMA_15'].shift() * (1 + pct_15))

    for na in range(len(merged_df['SMA_30'].isna())):
        merged_df['SMA_30'] = merged_df['SMA_30'].fillna(merged_df['SMA_30'].shift() * (1 + pct_30))

    merged_df['MACD'] = merged_df['EMA_12'] - merged_df['EMA_26']
    merged_df['MACD_SIGNAL'] = merged_df['MACD'].ewm(span=9, adjust=False).mean()  # Signal line

    merged_df = merged_df.drop(merged_df.index[:43])
    merged_df.dropna(subset = 'lag_5', inplace = True)  


    ## SEPARANDO TREINO PARA A SELEÇÃO DE FEATURES VIA LASSO
    # ## Separação treino
    train_size = int(len(merged_df)*0.8)
    train_df = merged_df.iloc[:train_size]
    
    # Scaling das Fatures do Modelo Lasso
    scaler = MinMaxScaler()
    train_scaled = scaler.fit_transform(train_df.drop(col, axis=1))

    # Convert the scaled data back to a DataFrame
    train_scaled_df = pd.DataFrame(train_scaled, columns=train_df.drop(col, axis=1).columns, index=train_df.index)

    # Merge the scaled features with the target variable
    train_scaled_df[col] = train_df[col]

    #test_scaled_df = test_scaled_df.dropna()

    # Split the scaled data into Features and Label
    y_train = train_scaled_df[col]
    X_train = train_scaled_df.drop(col, axis=1)

    # Parâmetros GridSearchCV
    params = {"alpha":np.logspace(-4, 4, num=100)}#0.00001, 10, 500)}

    # N Folds 
    kf = KFold(n_splits=5,shuffle=True, random_state=42)

    # Modelo Lasso
    lasso = Lasso(max_iter = 10000)

    # GridSearch
    lasso_cv = GridSearchCV(lasso, param_grid = params, cv = kf)
    lasso_cv.fit(X_train, y_train)
    print("Best Params {}".format(lasso_cv.best_params_))

    ## Nome das colunas
    names = merged_df.drop(col, axis=1).columns
    print("Column Names: {}".format(names.values))

    alpha_select = list(lasso_cv.best_params_.values())[0]

    lasso1 = Lasso(alpha = alpha_select)
    lasso1.fit(X_train, y_train)

    # Coeficientes absoutos 
    lasso1_coef = np.abs(lasso1.coef_)

    # Plot
    plt.bar(names, lasso1_coef)
    plt.xticks(rotation=90)
    plt.grid()
    plt.title("Feature Selection Based on Lasso")
    plt.xlabel("Features")
    plt.ylabel("Importance")
    #plt.ylim(0, 100)
    plt.show()

    ## Criando features de exclusão
    laglist = ['lag_5','lag_7', 'lag_10', 'lag_15', 'lag_30','trend']
    techlist = ['EMA_9','EMA_12','EMA_26','SMA_5','SMA_15','SMA_30']
    MACDlist = ['MACD','MACD_SIGNAL']
    rsilist = ['RSI', 'RSI_lag_7','RSI_lag_5','RSI_lag_10','RSI_lag_15','RSI_lag_30']

    lista_de_listas_exclusao = [laglist, techlist, MACDlist, rsilist] #EMAlist, SMAlist, MACDlist]

    # Excluindo features 0.01
    feature_subset = np.array(names)[lasso1_coef>0.01]
    print("Selected Features: {}".format(feature_subset))

    # Appendando AVIR 
    feature_select = np.append(feature_subset, col)
    print("Selected Cols: {}".format(feature_select))

    ## Loop de seleção das exclusões:
    droplist = np.array([])
    # Iniciando loop
    for exlist in lista_de_listas_exclusao:
        mask = np.isin(names, exlist)
        #print(mask)

        ex_select = np.array(names)[mask]
        coef_select = np.array(lasso1_coef)[mask]
        #print(ex_select)
        #print(coef_select)

        if coef_select.size > 0:
            max_index = np.argmax(coef_select)  # Índice do valor máximo
            max_value = coef_select[max_index]   # Valor máximo
            max_name = ex_select[max_index] 
        else:
            #pass
            max_name = []

        #print(max_name)

        ## Selecionado aqui o nomes que vamos droppa de names
        names_to_drop = np.delete(exlist, np.isin(exlist, max_name))
        #print(names_to_drop)

        droplist = np.append(droplist, names_to_drop)
        print(droplist)

    feature_select_excluded = np.delete(feature_select, np.isin(feature_select, droplist))

    merged_df = merged_df[feature_select_excluded]

    ## REPETINDO O SCALING E FAZENDO DE FATO O SPLIT TRAIN-TEST
    ## Separação treino test
    train_size = int(len(merged_df)*0.8)
    train_df, test_df = merged_df.iloc[:train_size], merged_df.iloc[train_size:]

    #test_df

    # Scale the features
    scaler = MinMaxScaler()
    train_scaled = scaler.fit_transform(train_df.drop(col, axis=1))
    test_scaled = scaler.transform(test_df.drop(col, axis=1))

    # Convert the scaled data back to a DataFrame
    train_scaled_df = pd.DataFrame(train_scaled, columns=train_df.drop(col, axis=1).columns, index=train_df.index)
    test_scaled_df = pd.DataFrame(test_scaled, columns=test_df.drop(col, axis=1).columns, index=test_df.index)

    # Merge the scaled features with the target variable
    train_scaled_df[col] = train_df[col]
    test_scaled_df[col] = test_df[col]

    forecast_df = test_scaled_df[test_scaled_df[col].isna()]

    test_scaled_df = test_scaled_df.dropna()

    # Split the scaled data into Features and Label
    y_train = train_scaled_df[col]
    X_train = train_scaled_df.drop(col, axis=1)
    y_test = test_scaled_df[col]
    X_test = test_scaled_df.drop(col, axis=1)

    #forecast_df ; X_forecast, df de forecat do XGBoost
    X_forecast = forecast_df.drop(col, axis=1)
    X_forecast

    ## Iniciando o modelo XGBoost ##

    # Criando o score MAPE:
    def mape(y_true, y_pred):
        return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

    mape_scorer = make_scorer(mape, greater_is_better=False)  # MAPE should be minimized

    ## Estimando os melhores hyperparametros para o modelo XGBoost
    parameters = {
    'n_estimators': [100, 200, 300, 400],
    'learning_rate': [0.001, 0.005, 0.01, 0.05],
    'max_depth': [8, 10, 12, 15],
    'gamma': [0.001, 0.005, 0.01, 0.02],
    #'alpha': [0.1, 0.5, 1],  # Regularização L1
    'lambda': [0.5, 1, 1.5] # Regularização L2
    }

    eval_set = [(X_train, y_train), (X_test, y_test)]
    model = xgb.XGBRegressor(objective='reg:squarederror', verbosity=1, random_state = 42, early_stopping_rounds = 20)
    #clf = GridSearchCV(model, parameters, scoring={'MAPE': mape_scorer}, refit = 'MAPE', cv=5)

    clf = RandomizedSearchCV(model, parameters, scoring='neg_mean_absolute_error', refit=True, cv=10, n_iter=50, random_state = 42)  # Random Search para um modelo menos lento (?) O pc sofre.

    clf.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)

    print(f'Best params: {clf.best_params_}')
    print(f'Best validation score = {clf.best_score_}')

    ## MODELO ESEMBLE XGB + LASSO + RIDGE + GBOOST + MLP
    base_estimators = [
    ('xgb', xgb.XGBRegressor(objective='reg:squarederror', **clf.best_params_, random_state = 42)),  # Best params from RandomizedSearchCV
    ('ridge', Ridge()),
    ('lasso', Lasso()),
    #('catboost', CatBoostRegressor(verbose=0, random_state=42)),
    #('adaboost', AdaBoostRegressor(random_state=42)),
    ('gboost', GradientBoostingRegressor(random_state=42)),
    #('rf', RandomForestRegressor(n_estimators=100, random_state=42)),
    ('mlp', MLPRegressor(hidden_layer_sizes = (8,16,8), max_iter = 2000, random_state=42)) 
    ]

    reg = StackingRegressor(
        estimators=base_estimators,
        final_estimator= xgb.XGBRegressor(),  
        cv = 10
    )

    reg.fit(X_train, y_train)

    y_pred = reg.predict(X_test)

    ## erros e avaliação do modelo

    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)

    r2 = r2_score(y_test, y_pred)
    evs = explained_variance_score(y_test, y_pred)

    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    mpe = np.mean((y_test - y_pred) / y_test) * 100


    print(f"Mean Squared Error (MSE): {mse}")
    print(f"Mean Absolute Error (MAE): {mae}")
    print(f"R2 Score: {r2}")
    print(f"Explained Variance Score: {evs}")
    print(f"Mean Absolute Percentage Error (MAPE): {mape}")
    print(f"Mean Percentage Error (MPE): {mpe}")
    
    y_forecast = reg.predict(X_forecast)

    # Plot final Predictions
    plt.figure(figsize=(12, 6), dpi=100)
    plt.plot(test_scaled_df.index, y_test, label='Real')
    plt.plot(test_scaled_df.index, y_pred, color='red', label='Predicted')
    plt.plot(forecast_df.index, y_forecast, color = 'blue', label = 'Forecast')

    mean_y_test = y_test.mean()
    # Linha horizontal
    plt.axhline(mean_y_test, color='grey', linestyle='--', label='Historical Mean')

    plt.title('Model Predictions vs Real Prices')
    plt.xlabel('Date')
    plt.ylabel('Stock Price')
    plt.legend()
    plt.show()
    
    ## Retorna nada se o mape estiver ruim demais. Ainda assim printa, para conferêcia
    if mape > 5:
        print(f'MAPE for {col} is {mape:.2f}, skipping this column.')
        return None  # Retorne None se o MAPE for maior que 6
    
    ## appendando os resultado em um result_df
    result_df = pd.DataFrame(y_forecast, columns=[col], index=forecast_df.index)
    return result_df

In [ ]:
## Crando o df de forecasts para acumular em loop, vazio.

all_forecasts = pd.DataFrame()

In [ ]:
import concurrent.futures

# ProcessPoolExecutor
with concurrent.futures.ProcessPoolExecutor() as executor:
    futures = {executor.submit(stock_predict, col, df): col for col in df.columns}

    for future in concurrent.futures.as_completed(futures):
        col = futures[future]
        try:
            result_df = future.result()
            all_forecasts = pd.concat([all_forecasts, result_df], axis=1)
        except Exception as e:
            print(f'Error processing {col}: {e}')

all_forecasts.columns = df.columns

print(all_forecasts)

In [ ]:
## Fazendo o concat. 

df_final = all_forecasts.copy()

df_final.tail(10)

In [ ]:
df_pct = df_final.pct_change(periods = 5)*100

last_var = df_pct.iloc[-1]

top_5 = last_var.nlargest(10).index.tolist()

print(top_5)

df_select = df_final[top_5]

df_select.tail(10)